In [1]:
from pathlib import Path

def load_individual_parquet_files(spark, folder="../data/cleaned"):
    cleaned_path = Path(folder).resolve()
    parquet_files = sorted(cleaned_path.glob("*.parquet"))
    
    if not parquet_files:
        print(f"No Parquet files found in: {cleaned_path}")
        return []
    
    print(f"Found {len(parquet_files)} Parquet files:")
    for f in parquet_files:
        print(f" - {f.name}")
    
    return parquet_files

# Example usage
parquet_files = load_individual_parquet_files(None)


Found 8 Parquet files:
 - part-00000-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
 - part-00001-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
 - part-00002-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
 - part-00003-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
 - part-00004-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
 - part-00005-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
 - part-00006-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
 - part-00007-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet


In [4]:
from pyspark.sql import SparkSession
from pathlib import Path

spark = SparkSession.builder \
    .appName("FareTipModeling") \
    .master("local[*]") \
    .getOrCreate()


In [5]:
folder_path = Path(r"D:\L4S2\Big Data\assignment\NYC_Taxi_Trip_Data_Analysis\data\cleaned")

parquet_files = sorted(folder_path.glob("*.parquet"))
print(f"Found {len(parquet_files)} parquet files:")
for f in parquet_files:
    print(f.name)


Found 8 parquet files:
part-00000-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
part-00001-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
part-00002-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
part-00003-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
part-00004-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
part-00005-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
part-00006-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
part-00007-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet


In [6]:
def filter_valid_fares_tips(df):
    filtered_df = df.filter((df.fare_amount > 0) & (df.tip_amount >= 0))
    print(f"Filtered from {df.count()} rows to {filtered_df.count()} rows with valid fare and tip amounts")
    return filtered_df


In [7]:
for file_path in parquet_files:
    print(f"\nProcessing file: {file_path.name}")
    df = spark.read.parquet(str(file_path))
    
    # Run your filter
    filtered_df = filter_valid_fares_tips(df)
    
    # You can add more processing steps here (e.g. modeling)



Processing file: part-00000-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
Filtered from 3918134 rows to 3918134 rows with valid fare and tip amounts

Processing file: part-00001-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
Filtered from 3933455 rows to 3933455 rows with valid fare and tip amounts

Processing file: part-00002-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
Filtered from 3928190 rows to 3928190 rows with valid fare and tip amounts

Processing file: part-00003-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
Filtered from 3936382 rows to 3936382 rows with valid fare and tip amounts

Processing file: part-00004-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
Filtered from 3930808 rows to 3930808 rows with valid fare and tip amounts

Processing file: part-00005-c267671b-1832-48c2-97b7-10a9827a836f-c000.snappy.parquet
Filtered from 5205187 rows to 5205187 rows with valid fare and tip amounts

Processing file: part-00006-c2676

In [8]:
from pyspark.sql.functions import when, col

df = df.withColumn("label", when(col("tip_amount") > 0.5, 1).otherwise(0))
print(f"Label distribution:")
df.groupBy("label").count().show()


Label distribution:
+-----+-------+
|label|  count|
+-----+-------+
|    1|3060278|
|    0| 826338|
+-----+-------+



In [9]:
from pyspark.ml.feature import VectorAssembler

feature_cols = ["fare_amount", "trip_distance", "trip_time_in_secs"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

data = assembler.transform(df).select("features", "label")


IllegalArgumentException: [FIELD_NOT_FOUND] No such struct field `trip_time_in_secs` in `VendorID`, `tpep_pickup_datetime`, `tpep_dropoff_datetime`, `passenger_count`, `trip_distance`, `RatecodeID`, `store_and_fwd_flag`, `PULocationID`, `DOLocationID`, `payment_type`, `fare_amount`, `extra`, `mta_tax`, `tip_amount`, `tolls_amount`, `improvement_surcharge`, `total_amount`, `congestion_surcharge`, `Airport_fee`, `label`. SQLSTATE: 42704